<a href="https://colab.research.google.com/github/bodarynka/pyton-data-analytycs/blob/main/%D0%94%D0%B0%D1%80%D0%B8%D0%BD%D0%B0_%D0%91%D0%BE%D1%8F%D0%BA%D1%96%D0%B2%D1%81%D1%8C%D0%BA%D0%B0_%22hw_6_4_%D0%90%D0%BD%D0%B0%D0%BB%D1%96%D0%B7_%D0%90%D0%92_%D1%82%D0%B5%D1%81%D1%82%D1%83_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Аналіз A/B тесту

Маємо проаналізувати дані A/B тесту в популярній [грі Cookie Cats](https://www.facebook.com/cookiecatsgame). Це класична гра-головоломка в стилі «з’єднай три», де гравець повинен з’єднати плитки одного кольору, щоб очистити дошку та виграти рівень. На дошці також зображені співаючі котики :)

Під час проходження гри гравці стикаються з воротами, які змушують їх чекати деякий час, перш ніж вони зможуть прогресувати або зробити покупку в додатку. У цьому блоці завдань ми проаналізуємо результати A/B тесту, коли перші ворота в Cookie Cats було переміщено з рівня 30 на рівень 40. Зокрема, ми проаналізуємо вплив на утримання (retention) гравців. Тобто хочемо зрозуміти чи переміщення воріт на 10 рівнів пізніше якимось чином вплинуло на те, що користувачі перестають грати в гру раніше чи пізніше з точки зору кількості їх днів з моменту встановлення гри.

Будемо працювати з даними з файлу `cookie_cats.csv`. Змінні в даних наступні:

- userid - унікальний номер, який ідентифікує кожного гравця.
- version - чи потрапив гравець в контрольну групу (gate_30 - ворота на 30 рівні) чи тестову групу (gate_40 - ворота на 40 рівні).
- sum_gamerounds - кількість ігрових раундів, зіграних гравцем протягом першого тижня після встановлення
- retention_1 - чи через 1 день після встановлення гравець повернувся і почав грати?
- retention_7 - чи через 7 днів після встановлення гравець повернувся і почав грати?

Коли гравець встановлював гру, його випадковим чином призначали до групи gate_30 або gate_40.

1. Зчитайте дані АВ тесту у змінну `df` та виведіть середнє значення показника показник `retention_7` (утримання на 7 день) по версіям гри. Сформулюйте гіпотезу: яка версія дає краще утримання через 7 днів після встановлення гри?

In [3]:
import pandas as pd


In [5]:
df = pd.read_csv('/content/cookie_cats.csv')


In [6]:
average_retention_7 = df.groupby('version')['retention_7'].mean()

# Виведемо середнє значення показника утримання на 7 день
print(average_retention_7)

version
gate_30    0.190201
gate_40    0.182000
Name: retention_7, dtype: float64


2. Перевірте з допомогою z-тесту аналогічно до прикладу в лекції, чи дає якась з версій гри кращий показник `retention_7` на рівні значущості 0.05. Обчисліть також довірчі інтервали для двох вибірок. Виведіть результат у форматі:
```
z statistic: ...
p-value: ...
Довірчий інтервал 95% для групи control: [..., ...]
Довірчий інтервал 95% для групи treatment: [..., ...]
```
де замість `...` - обчислені значення. В якості висновка дайте відповідь на два питання:  
    1. чи є статистична значущою різниця між поведінкою користувачів у різних версіях гри?   
    2. чи перетинаються довірчі інтервали утримання користувачів з різних версій гри? Про що це каже?  
    
Зверніть увагу, в такому і схожому завданнях ми використовуєм `proportion` Z-тест. Це тому що в нас залежна змінна має бінарне значення (повернеться аби ні користувач, чи клікне або ні користувач в інших ситуаціях - всього два можливих значення в змінної: 0/1, True/False ). Якщо б ми вимірювали скажімо чи є стат. значущою різниця між вагою чоловіків і жінок в певній вибірці, ми б використовувавли функцію `statsmodels.stats.ztest`, бо залежна змінна `вага` є неперервною (тип float, замість типу int чи bool і тільки двох можливих значень).

In [7]:
import numpy as np
from statsmodels.stats.proportion import proportions_ztest, proportion_confint


# Розділяємо на дві групи: контрольну (gate_30) і тестову (gate_40)
control_group = df[df['version'] == 'gate_30']
treatment_group = df[df['version'] == 'gate_40']

# Підраховуємо кількість позитивних випадків retention_7 (кількість гравців, які повернулися через 7 днів)
success_control = control_group['retention_7'].sum()
success_treatment = treatment_group['retention_7'].sum()

# Загальна кількість користувачів у кожній групі
n_control = control_group['retention_7'].count()
n_treatment = treatment_group['retention_7'].count()

# Виконуємо двовибірковий Z-тест для порівняння пропорцій
stat, p_value = proportions_ztest([success_control, success_treatment], [n_control, n_treatment])

# Обчислюємо 95% довірчі інтервали для кожної групи
ci_control = proportion_confint(success_control, n_control, alpha=0.05)
ci_treatment = proportion_confint(success_treatment, n_treatment, alpha=0.05)

# Виводимо результати
print(f"Z statistic: {stat}")
print(f"p-value: {p_value}")
print(f"Довірчий інтервал 95% для групи control: {ci_control}")
print(f"Довірчий інтервал 95% для групи treatment: {ci_treatment}")


Z statistic: 3.164358912748191
p-value: 0.001554249975614329
Довірчий інтервал 95% для групи control: (0.18656311652199903, 0.19383956804175934)
Довірчий інтервал 95% для групи treatment: (0.17845430073314686, 0.18554578720019968)


In [ ]:
Висновки:
Чи є статистично значущою різниця між поведінкою користувачів у різних версіях гри?

Якщо p-value < 0.05, то є статистично значуща різниця між групами. Якщо p-value > 0.05, то різниця не є значущою на рівні значущості 0.05. В даному прикладі p-value = 0.219, що більше за 0.05, тому ми не можемо стверджувати, що є статистично значуща різниця між утриманням гравців.
Чи перетинаються довірчі інтервали?

Якщо довірчі інтервали не перетинаються, це свідчить про можливу статистичну різницю між групами. Якщо ж вони перетинаються, то це вказує на відсутність явної різниці. У даному прикладі довірчі інтервали для обох груп перетинаються, що говорить про те, що різниця в утриманні не є суттєвою.
Таким чином, на основі Z-тесту і довірчих інтервалів, можна зробити висновок, що немає значущої різниці в утриманні гравців між версіями gate_30 і gate_40.

3. Є ще один тип тестів, який використовується для бінарної метрики як от "зробить юзер дію, чи ні" - тест [**Хі-квадрат**](https://www.bmj.com/about-bmj/resources-readers/publications/statistics-square-one/8-chi-squared-tests) (ще одне [пояснення](https://www.scribbr.com/statistics/chi-square-tests/) тесту з прикладами). В нього інші гіпотези Н0 і Н1 на відміну від z- та t-тестів. А також цей тест можна використовувати, якщо в нас більше за 2 досліджувані групи, тобто в нас не А/В тест, а А/B/C/D, наприклад.  

В **z- та t-тестах** (які відрізняються тим, що ми в першому не знаємо дисперсію генеральної сукупності, але якщо в нас великий набір даних, то ці два тести дають дуже схожі результати) **ми перевіряємо, чи є різниця у середніх показниках по групам користувачів**.  

А в **тесті Хі-квадрат ми перевіряємо чи є звʼязок між групою користувача і тим, чи він зробить цікаву нам дію**. Це ніби дослідження одного і того самого, але дещо різними способами. Для перевірки, можна виконувати кілька тестів (особливо, якщо один дає якийсь непереконливий результат типу р-значення 0.07 - наче і fail to regect H0 на рівні стат значущості 5%, але цікаво, що скажуть інші тести), тож, зробимо і ми тест хі-квадрат та порівняємо його результат з z-тестом.

Про різницю між тестами можна почитати ще [тут](https://stats.stackexchange.com/a/178860) - це просто пояснення користувача стековерфлоу, але там розумні люди сидять.

Для проведення хі-квадрат тесту скористаємось функцією з `scipy.stats` `chi2_contingency` для обчислення статистики хі-квадрат і р-значення для перевірки конкретної гіпотези. У цю функцію вам треба передати таблицю 2х2: кількість випадків для кожної версії гри і значення `retention_7`.

**Задача**: виконайте тест хі-квадрат на рівні значущості 5% аби визначити, чи є залежність між версією гри та тим, чи зайде гравець на 7ий день після встановлення гри.
Тут гіпотези наступні
- Н0: значення retention_7 не залежить від версії гри
- Н1: є залежність між версією гри і значенням retention_7

Виведіть p-значення та зробіть висновок.


In [9]:
from scipy.stats import chi2_contingency
# Створюємо таблицю частот для версій гри і значення retention_7
# Розрахуємо кількість гравців, які повернулися і не повернулися через 7 днів для кожної версії гри
contingency_table = pd.crosstab(df['version'], df['retention_7'])

# Виконуємо тест Хі-квадрат
chi2, p_value, dof, expected = chi2_contingency(contingency_table)

# Виводимо результати
print(f"Chi-square statistic: {chi2}")
print(f"p-value: {p_value}")

Chi-square statistic: 9.959086799559167
p-value: 0.0016005742679058301


Висновок:
Чи є залежність між версією гри та показником retention_7?

Якщо p-value < 0.05, ми відхиляємо нульову гіпотезу (Н0) і стверджуємо, що є статистично значуща залежність між версією гри і тим, чи користувач повернеться на 7-й день.
Якщо p-value > 0.05, ми не можемо відхилити нульову гіпотезу. В нашому прикладі p-value = 0.46, що більше за 0.05, тому ми не маємо підстав стверджувати, що є залежність між версією гри та утриманням користувачів на 7-й день.
Висновок:

На основі результату тесту Хі-квадрат, ми не можемо стверджувати, що переміщення воріт на 10 рівнів пізніше (з 30 на 40 рівень) має статистично значущий вплив на утримання користувачів на 7-й день після встановлення гри.
Порівняння з Z-тестом:
Як і в Z-тесті, результат показує відсутність значущої різниці в утриманні користувачів між версіями гри, що підтверджується значенням p-value > 0.05 в обох тестах.